In [1]:
import random

#Pennylane
import pennylane as qml
from pennylane import numpy as np

#PyTorch
import torch
import torchvision
from torchvision import datasets, transforms

In [2]:
dev = qml.device('default.qubit', wires = 6)
n_qubits = 6
n_layer_steps = 5
n_layers_to_add = 2

In [3]:
def set_random_gates(n_qubits: int):
    
    gate_set = [qml.RX, qml.RY, qml.RZ]
    chosen_gates = []
    for i in range(n_qubits):
        chosen_gate = random.choice(gate_set)
        chosen_gates.append(chosen_gate)
    return chosen_gates

In [4]:
@qml.template
def apply_layer(gates, weights):
    
    for i in range(n_qubits):
        gates[i](weights[i], wires = i)
    
    tuples = [(i,i+1) for i in range(n_qubits-1)]

    for tup in tuples:
        qml.CZ(wires=[tup[0], tup[1]])

In [5]:
layer_gates = []
layer_weights = []
weights_1 = np.random.rand(n_qubits)*2*np.pi
gates_1 = set_random_gates(n_qubits)
weights_2 = np.random.rand(n_qubits)*2*np.pi
gates_2 = set_random_gates(n_qubits)
layer_weights.append(weights_1)
layer_weights.append(weights_2)
layer_gates.append(gates_1)
layer_gates.append(gates_2)

@qml.template #This could be a template for the setting the previous layers.
def frozen_circuit(layer_gates, layer_weights): #Must depend in the new parameters, 12 parameters, 6 per layer
    for i in range(len(layer_gates)):
        apply_layer(layer_gates[i], layer_weights[i])

In [6]:
print(layer_gates)
print(layer_weights)

[[<class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RX'>, <class 'pennylane.ops.qubit.RX'>, <class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RY'>], [<class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RZ'>, <class 'pennylane.ops.qubit.RZ'>, <class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RX'>]]
[tensor([4.25251137, 2.21357348, 3.74944973, 0.44930823, 3.33840587,
        6.10150727], requires_grad=True), tensor([3.69591492, 4.84322607, 5.86948628, 1.46705465, 0.47329481,
        4.91545338], requires_grad=True)]


In [7]:
frozen_circuit(layer_gates, layer_weights)

[RY(tensor(4.25251137, requires_grad=True), wires=[0]),
 RY(tensor(2.21357348, requires_grad=True), wires=[1]),
 RX(tensor(3.74944973, requires_grad=True), wires=[2]),
 RX(tensor(0.44930823, requires_grad=True), wires=[3]),
 RY(tensor(3.33840587, requires_grad=True), wires=[4]),
 RY(tensor(6.10150727, requires_grad=True), wires=[5]),
 CZ(wires=[0, 1]),
 CZ(wires=[1, 2]),
 CZ(wires=[2, 3]),
 CZ(wires=[3, 4]),
 CZ(wires=[4, 5]),
 RY(tensor(3.69591492, requires_grad=True), wires=[0]),
 RY(tensor(4.84322607, requires_grad=True), wires=[1]),
 RZ(tensor(5.86948628, requires_grad=True), wires=[2]),
 RZ(tensor(1.46705465, requires_grad=True), wires=[3]),
 RY(tensor(0.47329481, requires_grad=True), wires=[4]),
 RX(tensor(4.91545338, requires_grad=True), wires=[5]),
 CZ(wires=[0, 1]),
 CZ(wires=[1, 2]),
 CZ(wires=[2, 3]),
 CZ(wires=[3, 4]),
 CZ(wires=[4, 5])]

In [8]:
layer_gates = []
layer_weights = []

In [9]:
weights = np.random.rand(n_qubits)*2*np.pi
gates = set_random_gates(n_qubits)
print(weights)
print(gates)
apply_layer(gates, weights)

[4.54133674 4.26672475 5.65155347 3.80294482 5.61712966 1.87108463]
[<class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RZ'>, <class 'pennylane.ops.qubit.RZ'>, <class 'pennylane.ops.qubit.RZ'>, <class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RZ'>]


[RY(tensor(4.54133674, requires_grad=True), wires=[0]),
 RZ(tensor(4.26672475, requires_grad=True), wires=[1]),
 RZ(tensor(5.65155347, requires_grad=True), wires=[2]),
 RZ(tensor(3.80294482, requires_grad=True), wires=[3]),
 RY(tensor(5.61712966, requires_grad=True), wires=[4]),
 RZ(tensor(1.87108463, requires_grad=True), wires=[5]),
 CZ(wires=[0, 1]),
 CZ(wires=[1, 2]),
 CZ(wires=[2, 3]),
 CZ(wires=[3, 4]),
 CZ(wires=[4, 5])]

In [10]:
for step in range(n_layer_steps):
    
    for prev_layers in layer_gates:
        apply_layer(prev_layers, prev_weights)
        
    for layer in range(n_layers_to_add):
        set_gates = set_random_gates(n_qubits)

    #train circuit
    
    #obtain weights and add them properly to layer_weights